In [ ]:
import mmcv
from mmcv import Config
from mmseg.apis import inference_segmentor, init_segmentor
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from mmseg.datasets import build_dataloader, build_dataset
from sklearn.decomposition import PCA

In [ ]:
# initialize the segmentor
cfg = Config.fromfile('configs/segformer/segformer_mit-b0_8x1_1024x1024_160k_cityscapes.py')
checkpoint_file = 'checkpoint/segformer_mit-b0_8x1_1024x1024_160k_cityscapes_20211208_101857-e7f88502.pth'
model = init_segmentor(cfg, checkpoint_file, device='cuda:0')

# load the dataset
data_dir = 'data/cityscapes/leftImg8bit/train'
#img_infos = mmcv.scan_images(data_dir, recursive=True)
#dataset = build_dataset(cfg.data.train)
dataset = build_dataset(cfg.data.val)#改为验证集后，就不会裁剪了，那个encode_decode的尺寸会是原尺寸（1，19，1024，2048）
dataloader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=1,
    dist=False,
    shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model.eval()
features = {}
targets = {}
model.to(device)
feat_maps = np.empty([0, 19,1024,1024])
for i, data in enumerate(dataloader):
  #if i == 1:
  #  break
  img_metas = data['img_metas'].data
  images = data['img'].data[0].to(device)
  #print(images.shape)#torch.Size([1, 3, 1024, 1024])
  #target = data['gt_semantic_seg'].data[0].to(device)
  #print(target)
  #print(target.shape)#torch.Size([1, 1, 1024, 1024])
  #img_path = data['img_metas'].data[0][0].get('filename')
  #feat = model.extract_feat(images)
  feat_map = model.encode_decode(images, img_metas)
  #feat_map = out.squeeze()
  
  feat_map = feat_map.cpu().detach().numpy()
  #feat_map = (feat_map - np.mean(feat_map, axis=0)) / np.std(feat_map,axis=0)
  #feat_map = feat_map.reshape(19,-1)
  feat_maps = np.append(feat_maps, feat_map, axis=0)
  #print("out.shape:", out.shape)#out.shape: torch.Size([1, 19, 1024, 1024])
  #print("out:", out)

  #print('feat[0].shape:',feat[0].shape)#feat[0].shape: torch.Size([1, 32, 256, 256])
  #print('feat[1].shape:',feat[1].shape)#feat[1].shape: torch.Size([1, 64, 128, 128])
  #print('feat[2].shape:',feat[2].shape)#feat[2].shape: torch.Size([1, 160, 64, 64])
  #print('feat[3].shape:',feat[3].shape)#feat[3].shape: torch.Size([1, 256, 32, 32])
  #print(model.num_classes)
  #output = inference_segmentor(model, img_path)
  #print(output)
  #result_flat = output[0].flatten()

  # tsne = TSNE(n_components=2, random_state=0)
  # result_tsne = tsne.fit_transform(result_flat.reshape(-1, 1))
  # plt.scatter(result_tsne[:, 0], result_tsne[:, 1], c=result_flat, cmap='tab20')
  # plt.show()
  #print(output[0].shape)#(1024, 2048)
  #print(features['train'].shape)
  #features['train'] = np.append(features['train'],output[0], axis=0)
  #targets['train'] = np.append(targets['train'],target.cpu(), axis=0)
  #print(features['train'].shape)
  if i == 200:
    break

In [ ]:
from sklearn.preprocessing import StandardScaler
tsne = TSNE(n_components=2,random_state=123)

In [ ]:
for label in range(19):
    features = feat_maps[:,label,:,:]
    flatten_feat =[]
    for f in features:
        flatten_feat.append(f.reshape(-1))
    X = np.array(flatten_feat)
    X = StandardScaler().fit_transform(X)
    pca_50 = PCA(n_components=32)

    pca_result_50 = pca_50.fit_transform(X)
    X_tsne = tsne.fit_transform(pca_result_50)

    data.append((X_tsne, label))

In [ ]:
PALETTE = np.array([[128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156],
            [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0],
            [107, 142, 35], [152, 251, 152], [70, 130, 180], [220, 20, 60],
            [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100],
            [0, 80, 100], [0, 0, 230], [119, 11, 32]])
colors  = ['#%02x%02x%02x' % (r,g,b) for r,g,b in PALETTE]

for X_tsne, label in data:
    label_color = colors[label]
    plt.scatter(X_tsne[:,0], X_tsne[:,1], c=label_color)

plt.show()